In [1]:
import sqlite3
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
file_path = '../data/tv/tv_live_1.csv'

In [11]:
df_tv = pd.read_csv(file_path, delimiter=';', thousands=' ', decimal=',', parse_dates=['Date'])
df_tv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 972606 entries, 0 to 972605
Data columns (total 23 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Month                                       972606 non-null  int64  
 1   Week                                        972606 non-null  int64  
 2   Weekday                                     972606 non-null  object 
 3   Date                                        972606 non-null  object 
 4   Time                                        972606 non-null  object 
 5   Dayparts                                    918809 non-null  object 
 6   Channel Groups                              972606 non-null  object 
 7   Channel                                     972606 non-null  object 
 8   PIB pos                                     972606 non-null  int64  
 9   PIB (real) rel                              972606 non-null  object 
 

In [12]:
df_tv.head()

,Month,Week,Weekday,Date,Time,Dayparts,Channel Groups,Channel,PIB pos,PIB (real) rel,PIB count,"Dur rounded,sp",Spot Class,Block Code,Syndicate,Producer,Brand,Film Code,Prog Campaign,Prog Before,Prog After,Film Code/2,TRP All 40+ cities 200- (i wsie) Nat[TSV2]
0,202001,202001,Wed,01.01.2020,02:00:05,Off stacji,TVP tem,TVP Seriale,1,First,10,30,Commercial,Between,Media expert sp z oo,Media expert sp z oo,MEDIA EXPERT,Media expert sklep 1599zl czyszczenie magazyno...,PILKARSKI POKER /FILM POL./,NaN,Reklama dzieciom - final akcji,259128,0.05
1,202001,202001,Wed,01.01.2020,02:02:33,Off stacji,Polsat tem,Fokus TV,1,First,10,30,Commercial,Between,Media expert sp z oo,Media expert sp z oo,MEDIA EXPERT,Media expert sklep 1599zl czyszczenie magazyno...,GWIAZDY LOMBARDU /REALITY SHOW/,Gwiazdy lombardu,Infokus,259134,0.41
2,202001,202001,Wed,01.01.2020,02:03:59,Off stacji,TVP tem,TVP Info,4,2nd last,5,30,Commercial,Between,Media expert sp z oo,Media expert sp z oo,MEDIA EXPERT,Media expert sklep 1599zl czyszczenie magazyno...,WIADOMOSCI WYDANIE SPECJALNE,Wiadomosci wydanie specjalne,Minela dwudziesta,259128,0.38
3,202001,202001,Wed,01.01.2020,02:04:16,Off stacji,Polsat tem,Eska TV [8TV],1,First,8,30,Commercial,Within,Media expert sp z oo,Media expert sp z oo,MEDIA EXPERT,Media expert sklep 1599zl czyszczenie magazyno...,HITY NA CZASIE /PROG.MUZ./,Hity na czasie,Hity na czasie,259114,0.08
4,202001,202001,Wed,01.01.2020,02:05:10,Off stacji,Polsat tem,Stopklatka,1,First,10,30,Commercial,Within,Media expert sp z oo,Media expert sp z oo,MEDIA EXPERT,Media expert sklep 1599zl czyszczenie magazyno...,DROGOWKA /FILM DRAM.KRYM.POL./,Drogowka,Drogowka,259128,0.36
